# 02 - Data from the Web

## Deadline
Wednesday October 25, 2017 at 11:59PM

## Important Notes
* Make sure you push on GitHub your Notebook with all the cells already evaluated (i.e., you don't want your colleagues to generate unnecessary Web traffic during the peer review)
* Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you plan to implement!
* Please write all your comments in English, and use meaningful variable names in your code.

## Background
In this homework we will extract interesting information from www.topuniversities.com and www.timeshighereducation.com, two platforms that maintain a global ranking of worldwide universities. This ranking is not offered as a downloadable dataset, so you will have to find a way to scrape the information we need!
You are not allowed to download manually the entire ranking -- rather you have to understand how the server loads it in your browser. For this task, Postman with the Interceptor extension can help you greatly. We recommend that you watch this [brief tutorial](https://www.youtube.com/watch?v=jBjXVrS8nXs&list=PLM-7VG-sgbtD8qBnGeQM5nvlpqB_ktaLZ&autoplay=1) to understand quickly how to use it.

## Assignment
1. Obtain the 200 top-ranking universities in www.topuniversities.com ([ranking 2018](https://www.topuniversities.com/university-rankings/world-university-rankings/2018)). In particular, extract the following fields for each university: name, rank, country and region, number of faculty members (international and total) and number of students (international and total). Some information is not available in the main list and you have to find them in the [details page](https://www.topuniversities.com/universities/ecole-polytechnique-fÃ©dÃ©rale-de-lausanne-epfl).
Store the resulting dataset in a pandas DataFrame and answer the following questions:
- Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?
- Answer the previous question aggregating the data by (c) country and (d) region.

Plot your data using bar charts and describe briefly what you observed.

2. Obtain the 200 top-ranking universities in www.timeshighereducation.com ([ranking 2018](http://timeshighereducation.com/world-university-rankings/2018/world-ranking)). Repeat the analysis of the previous point and discuss briefly what you observed.

3. Merge the two DataFrames created in questions 1 and 2 using university names. Match universities' names as well as you can, and explain your strategy. Keep track of the original position in both rankings.

4. Find useful insights in the data by performing an exploratory analysis. Can you find a strong correlation between any pair of variables in the dataset you just created? Example: when a university is strong in its international dimension, can you observe a consistency both for students and faculty members?

5. Can you find the best university taking in consideration both rankings? Explain your approach.

Hints:
- Keep your Notebook clean and don't print the verbose output of the requests if this does not add useful information for the reader.
- In case of tie, use the order defined in the webpage.



In [1]:
# Import libraries
# to install selenium in Anaconda type in bash the following line: 
# $ conda install -c conda-forge selenium
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [2]:
# Make the request
URL = 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/'\
                +'world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'
r = requests.get(URL)
print('r = {r} // status_code = {status}'.format(r=r,status=r.status_code))
r.content
soupp = BeautifulSoup(r.content,'html.parser')
rank_json = json.loads(r.text)
rank_df = pd.DataFrame()
rank_df = rank_df.from_dict(rank_json['data']).head(200)
rank_df = rank_df[['name', 'rank', 'stats_number_students', 'stats_student_staff_ratio', 'stats_pc_intl_students']]
rank_df.head()

r = <Response [200]> // status_code = 200


,name,rank,stats_number_students,stats_student_staff_ratio,stats_pc_intl_students
0,University of Oxford,1,"20,409",11.2,38%
1,University of Cambridge,2,"18,389",10.9,35%
2,California Institute of Technology,=3,"2,209",6.5,27%
3,Stanford University,=3,"15,845",7.5,22%
4,Massachusetts Institute of Technology,5,"11,177",8.7,34%


In [3]:
rank_df.dtypes

name                         object
rank                         object
stats_number_students        object
stats_student_staff_ratio    object
stats_pc_intl_students       object
dtype: object

In [4]:
rank_df.stats_pc_intl_students = rank_df.stats_pc_intl_students.str.replace('%', '').astype('double')/100
rank_df.stats_number_students = rank_df.stats_number_students.str.replace(',', '').astype('int')
rank_df.stats_student_staff_ratio=rank_df.stats_student_staff_ratio.astype(float)
rank_df['fac_memb_int'] = -99
rank_df['fac_memb_tot'] = rank_df['stats_number_students']/rank_df['stats_student_staff_ratio']
rank_df['nb_stud_int'] = rank_df['stats_number_students']*rank_df['stats_pc_intl_students']
rank_df['fac_memb_tot'] = rank_df['fac_memb_tot'].astype(int)
rank_df['nb_stud_int'] = rank_df['nb_stud_int'].astype(int)
rank_df['stats_staff_student_ratio'] = 1/rank_df['stats_student_staff_ratio']
rank_df=rank_df.drop('stats_student_staff_ratio', axis=1)
rank_df['rank'] = rank_df['rank'].astype(str)
rank_df['rank'] = rank_df['rank'].replace({r'=': ''}, regex=True).apply(pd.to_numeric).astype(int)

In [6]:
rank_df.head()

,name,rank,stats_number_students,stats_pc_intl_students,fac_memb_int,fac_memb_tot,nb_stud_int,stats_staff_student_ratio
0,University of Oxford,1,20409,0.38,-99,1822,7755,0.089286
1,University of Cambridge,2,18389,0.35,-99,1687,6436,0.091743
2,California Institute of Technology,3,2209,0.27,-99,339,596,0.153846
3,Stanford University,3,15845,0.22,-99,2112,3485,0.133333
4,Massachusetts Institute of Technology,5,11177,0.34,-99,1284,3800,0.114943


In [7]:
rank_df.dtypes

name                          object
rank                           int64
stats_number_students          int64
stats_pc_intl_students       float64
fac_memb_int                   int64
fac_memb_tot                   int64
nb_stud_int                    int64
stats_staff_student_ratio    float64
dtype: object

In [10]:
list1 = list(rank_df['name'])


AttributeError: 'list' object has no attribute 'dtype'